In [1]:
# !pip install geemap

<h1><center>Monitoring Grassland Health in Mongolia </center></h1>

**Steps to create soum level change detection map:**

1. Choose Province
2. Choose Soum
3. Choose satellite derived index (more about these indices: https://www.usgs.gov/landsat-missions/landsat-surface-reflectance-derived-spectral-indices)
4. Select Year
5. Check `Only Pasture` (This will display areas that are deemed to be only grazing lands)
6. Check `Apply fmask(remove cloud, shadow, snow)` (This will use only images without any cloud, shadow)

**How to intrepret the map?**

The algorithm display vegetation change map for a given year for a given soum. The color maps can be intrepreted in relative to the mean of 8 years between 2014-2021. In other words, a color map indicates how much given year's vegetation deviate from from the mean value between 2014-2021. The three colors - green, yellow, red - represents the standard deviation distance. Green displays values where distance from mean is positive, yellow neatrul, and red negative. Thus, the intensity of the color indicates the larger distance from the mean.    

**Web Apps:** https://nogoonzun.herokuapp.com/

**Contact:** Khusel Avirmed (ta346@cornell.edu)

In [2]:
# Check geemap installation
import subprocess

try:
    import geemap
except ImportError:
    print('geemap package is not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

In [3]:
# Import libraries
import os
import ee
import geemap
import ipywidgets as widgets
from bqplot import pyplot as plt
from ipyleaflet import WidgetControl

In [45]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import altair as alt
import numpy as np
import folium
# <VegaLite 4 object>
# alt.renderers.enable('default')

In [5]:
# try:
#         ee.Initialize()
# except Exception as e:
#         ee.Authenticate()
#         ee.Initialize()

In [6]:
# Map.remove_ee_layer('Landsat 8' + ': ' + str(year_widget.value) + ' ' + str(nd_indices.value))

In [7]:
# Create an interactive map
Map = geemap.Map()
# Add Earth Engine data
fc = ee.FeatureCollection('users/ta346/mng-bounds/soum_aimag')

In [8]:
Map.add_basemap('ROAD')
Map.addLayer(ee.Image().paint(fc, 0, 2), {'palette': 'black'}, 'mng soums')

# states = ee.FeatureCollection('TIGER/2018/States')
# Map.addLayer(states, {}, 'US States')
Map.centerObject(fc, 5)
Map

Map(center=[46.96522531802441, 103.14224362261275], controls=(WidgetControl(options=['position', 'transparent_…

In [9]:
# fc.first().getInfo()

In [10]:
# bring soum province names as list of dictionaries
import json
with open("soum_province_names.json", "r") as read_file:
    newdict = json.load(read_file)

In [11]:
# Designe interactive widgets

style = {'description_width': 'initial'}

output_widget = widgets.Output(layout={'border': '1px solid black'})
output_control = WidgetControl(widget=output_widget, position='topleft')
Map.add_control(output_control)

# admin1_widget = widgets.Text(
#     description='State:',
#     value='Tennessee',
#     width=200,
#     style=style
# )

dc = 'Arkhangai'
province = widgets.Dropdown(
    options = list(newdict),
    description = 'Province:',
    disabled = False,
)

soum = widgets.Dropdown(
    options=newdict[dc],
    value = "Chuluut",
    description = 'Soum:',
    disabled = False,
)

def on_value_change(change):
    dc = change.new
    soum.options = newdict[dc]

province.observe(on_value_change, 'value')

nd_options = ['Normalized Difference Vegetation Index (NDVI)', 
              'Normalized Difference Water Index (NDWI)',
              'Modified Soil Adjusted Vegetation Index (MSAVI)',
              'Enhanced Vegetation Index (EVI)',
              'Near-Infrared Reflectence Vegetation (NIRv)']

nd_indices = widgets.Dropdown(options=nd_options, 
                              value='Normalized Difference Vegetation Index (NDVI)', 
                              description='Satellite Derived Index:', 
                              style=style)

year_widget = widgets.IntSlider(min=2014, max=2021, value=2014, description='Selected year:', width=400, style=style)

pasture_widget = widgets.Checkbox(
    value=True,
    description='Only pasture',
    style=style
)

fmask_widget = widgets.Checkbox(
    value=True,
    description='Apply fmask(remove cloud, shadow, snow)',
    style=style
)

submit = widgets.Button(
    description='Submit',
    button_style='primary',
    tooltip='Click me',
    style=style
)

full_widget = widgets.VBox([
    widgets.VBox([province, soum, nd_indices, year_widget, pasture_widget, fmask_widget]),
    submit
])

full_widget

In [12]:
# states = fc.sort('aimag_eng', True)
# names = states.aggregate_array("aimag_eng").distinct().getInfo()

# newdict = {}
# for i in names:
#     dictionary = {
#         i : fc.filter(ee.Filter.eq("aimag_eng", i)).aggregate_array('soum_eng').distinct().getInfo()
#     }
    
#     newdict.update(dictionary)

# # write to json
# import json
# with open('soum_province_names.json', 'w') as fout:
#     json.dump(newdict , fout)

In [13]:
pasture = ee.Image("users/ta346/pasture_delineation/pas_raster_new")
# winter - 1, summer - 2, pasture-not-used - 3
pas = pasture # preserve all only pasture are. Pixels that don't have value of 0 in the datamask band (thos that are summer and winter pasture) get a value of 1
pas_winter = pasture.eq(1) # pixels that do have 1 in the image (those that are summer/fall pasture) will get value of 1, everything else 0
pas_summer = pasture.eq(2) # pixels that do have 2 in the image (those that are summer/fall pasture) will get value of 1, everything else 0

# define function to updateMask to apply to image collections
def mask_pas (img):
    return img.updateMask(pas).copyProperties(img, ['system:time_start'])
# define function to updateMask to apply to image collections
def mask_pas_winter (img):
    return img.updateMask(pas_winter).copyProperties(img, ['system:time_start'])
# define function to updateMask to apply to image collections
def mask_pas_summer (img):
    return img.updateMask(pas_summer).copyProperties(img, ['system:time_start'])

In [14]:
#ndvi function for image collections for landsat 8: C02 Coleection
def addNDVI_C2(im):
    ndvi = im.normalizedDifference(['SR_B4','SR_B3']).rename('ndvi').copyProperties(im, ['system:time_start'])
    return im.addBands(ndvi)

# soil adjusted vegetation index (SAVI) for landsat 8: C02 Coleection
def addSAVI_C2(image):
    savi = image.expression(
        '1.5 * (NIR - RED) / (NIR + RED + 0.5)', {
            'NIR': image.select('SR_B4'),
            'RED': image.select('SR_B3')}).rename('savi')
    return image.addBands(savi)

# ------Create an add MSAVI variable function for landsat 8: C02 Coleection
def addMSAVI_C2(image):
    msavi = image.expression(
        '(2 * NIR + 1 - sqrt(pow((2 * NIR + 1), 2) - 8*(NIR - RED)))/2', {
            'NIR': image.select('SR_B4'),
            'RED': image.select('SR_B3')}).rename('msavi')
    return image.addBands(msavi) #to find MSAVI2, divide by 2

# EVI for landsat 8: C02 Coleection
def addEVI_C2(im):
    evi = im.expression(
        '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
            'NIR': im.select('SR_B4'),
            'RED': im.select('SR_B3'),
            'BLUE': im.select('SR_B1')
            }).rename("evi")
    return im.addBands(evi)

# ------Create an add NIRv variable function for landsat 8: C02 Coleection
def addNIRv_C2(image):
    nirv = image.expression(
        'NIR * ((NIR - RED) / (NIR + RED))', {
            'NIR': image.select('SR_B4'),
            'RED': image.select('SR_B3')}).rename('nirv')
    return image.addBands(nirv)


# ------Create an add NDWI variable function for landsat 8: C02 Coleection
def addNDWI_C2(image):
    ndwi = image.expression(
        '(NIR - SWIR) / (NIR + SWIR)', {
            'NIR': image.select('SR_B4'),
            'SWIR': image.select('SR_B5')}).rename('ndwi')
    return image.addBands(ndwi)

# write function to download csv file from image collections
def yearlyComposite(startYear, endYear, shp):
    # function takes startYear, endYear, landcover mask (1985-1999: maskLC2000, 2000-2009: maskLC2010, 2010-2020: maskLC2020), shp = bag or soum, file name: str, folder_name: str (google drive folder to download to)
    startYear = startYear
    endYear = endYear
    stepList = ee.List.sequence(startYear, endYear)
    
    def func_qla(year):
        startDate = ee.Date.fromYMD(year,6,1)
        endDate = ee.Date.fromYMD(year,8,31)
        
        # Applies scaling factors.
        def applyScaleFactors(image):
            opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2).toFloat()
            thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0).toFloat()
            return image.addBands(opticalBands, overwrite = True) \
                      .addBands(thermalBands, overwrite = True)
        
        def func_ewa(img):
            dat = img.select(['SR_B2','SR_B3','SR_B4','SR_B5','SR_B6','SR_B7', 'QA_PIXEL'],
                             ['SR_B1','SR_B2','SR_B3','SR_B4','SR_B5','SR_B7', 'QA_PIXEL']).set('system:time_start', img.get('system:time_start'))
            
            qa = img.select('QA_PIXEL')

            # Bits 1 is diluted cloud; 3 cloud, and 5 cloud shadow
            dilatedCloud = (1 << 1)
            cloud = (1 << 3)
            cloudShadow = (1 << 4)

            # Both flags should be set to zero, indicating clear conditions.
            mask = qa.bitwiseAnd(dilatedCloud).eq(0) \
                        .And(qa.bitwiseAnd(cloud).eq(0)) \
                        .And(qa.bitwiseAnd(cloudShadow).eq(0))

            datMasked = dat.mask(mask)

            return datMasked
        
        ls8_collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
                                .filterBounds(shp) \
                                .filterDate(startDate, endDate) \
                                .map(applyScaleFactors) \
                                .map(func_ewa) \
                                .map(mask_pas)
        
        # apply indices functions and select only those bands
        ls8_collection_indices = ls8_collection.map(addNDVI_C2) \
                                            .map(addEVI_C2) \
                                            .map(addSAVI_C2) \
                                            .map(addMSAVI_C2) \
                                            .map(addNIRv_C2) \
                                            .map(addNDWI_C2) \
                                            .select('ndvi', 'evi', 'savi', 'msavi', 'nirv', 'ndwi')
        
        
        composite_i = ls8_collection_indices.median().clip(shp).set('system:time_start', startDate)
        
        return composite_i
    
    filterCollection = stepList.map(func_qla)

    yearlyComposites = ee.ImageCollection.fromImages(filterCollection) #return single image for each year
    
    return yearlyComposites

In [15]:
def create_reduce_region_function(geometry,
                                  reducer=ee.Reducer.mean(),
                                  scale=1000,
                                  crs='EPSG:4326',
                                  bestEffort=True,
                                  maxPixels=1e13,
                                  tileScale=4):
  """Creates a region reduction function.

  Creates a region reduction function intended to be used as the input function
  to ee.ImageCollection.map() for reducing pixels intersecting a provided region
  to a statistic for each image in a collection. See ee.Image.reduceRegion()
  documentation for more details.

  Args:
    geometry:
      An ee.Geometry that defines the region over which to reduce data.
    reducer:
      Optional; An ee.Reducer that defines the reduction method.
    scale:
      Optional; A number that defines the nominal scale in meters of the
      projection to work in.
    crs:
      Optional; An ee.Projection or EPSG string ('EPSG:5070') that defines
      the projection to work in.
    bestEffort:
      Optional; A Boolean indicator for whether to use a larger scale if the
      geometry contains too many pixels at the given scale for the operation
      to succeed.
    maxPixels:
      Optional; A number specifying the maximum number of pixels to reduce.
    tileScale:
      Optional; A number representing the scaling factor used to reduce
      aggregation tile size; using a larger tileScale (e.g. 2 or 4) may enable
      computations that run out of memory with the default.

  Returns:
    A function that accepts an ee.Image and reduces it by region, according to
    the provided arguments.
  """

  def reduce_region_function(img):
    """Applies the ee.Image.reduceRegion() method.

    Args:
      img:
        An ee.Image to reduce to a statistic by region.

    Returns:
      An ee.Feature that contains properties representing the image region
      reduction results per band and the image timestamp formatted as
      milliseconds from Unix epoch (included to enable time series plotting).
    """

    stat = img.reduceRegion(
        reducer=reducer,
        geometry=geometry,
        scale=scale,
        crs=crs,
        bestEffort=bestEffort,
        maxPixels=maxPixels,
        tileScale=tileScale)

    return ee.Feature(geometry, stat).set({'millis': img.date().millis()})
  return reduce_region_function

In [16]:
def create_reduce_regions_function(collection,
                                  reducer=ee.Reducer.mean(),
                                  scale=1000,
                                  crs='EPSG:4326',
                                  tileScale=4):
  """Creates multiple regions reduction function.

  Creates multiple regions reduction function intended to be used as the input function
  to ee.ImageCollection.map() for reducing pixels intersecting a provided region
  to a statistic for each image in a collection. See ee.Image.reduceRegions()
  documentation for more details.

  Args:
    collection:
      FeatureCollection - The features to reduce over
    reducer:
      Optional; An ee.Reducer that defines the reduction method.
    scale:
      Optional; A number that defines the nominal scale in meters of the
      projection to work in.
    crs:
      Optional; An ee.Projection or EPSG string ('EPSG:5070') that defines
      the projection to work in.
    tileScale:
      Optional; A number representing the scaling factor used to reduce
      aggregation tile size; using a larger tileScale (e.g. 2 or 4) may enable
      computations that run out of memory with the default.

  Returns:
    A function that accepts an ee.Image and reduces it by region, according to
    the provided arguments.
  """

  def reduce_regions_function(img):
    """Applies the ee.Image.reduceRegion() method.

    Args:
      img:
        An ee.Image to reduce to a statistic by region.

    Returns:
      An ee.Feature that contains properties representing the image region
      reduction results per band and the image timestamp formatted as
      milliseconds from Unix epoch (included to enable time series plotting).
    """

    stat = img.reduceRegions(
        reducer=reducer,
        collection=collection,
        scale=scale,
        crs=crs,
        tileScale=tileScale)

    return ee.Feature(collection, stat).set({'millis': img.date().millis()})
  return reduce_regions_function

In [17]:
# Define a function to transfer feature properties to a dictionary.
def fc_to_dict(fc):
    prop_names = fc.first().propertyNames()
    prop_lists = fc.reduceColumns(
        reducer=ee.Reducer.toList().repeat(prop_names.size()),
        selectors=prop_names).get('list')
    return ee.Dictionary.fromLists(prop_names, prop_lists)

In [18]:
# Function to add date variables to DataFrame.
def add_date_info(df):
    df['Timestamp'] = pd.to_datetime(df['millis'], unit='ms')
    df['Year'] = pd.DatetimeIndex(df['Timestamp']).year
    df['Month'] = pd.DatetimeIndex(df['Timestamp']).month
    df['Day'] = pd.DatetimeIndex(df['Timestamp']).day
    df['DOY'] = pd.DatetimeIndex(df['Timestamp']).dayofyear
    return df

In [19]:
# create date range from today to past 20 years
today = ee.Date(pd.to_datetime('today'))
date_range = ee.DateRange(today.advance(-20, 'years'), today)

In [20]:
# Click event handler
def submit_clicked(b):
    
    try:
        Map.remove_ee_layer('Landsat 8' + ': ' + str(nd_indices.value))
    except Exception as e:
        print(e)
        
    with output_widget:
        
        output_widget.clear_output()
        print('Computing...')
        Map.default_style = {'cursor': 'wait'}
        
        try:
            province_id = province.value 
            soum_id = soum.value
            nd_indices_id = nd_indices.value
            selected_year = year_widget.value 
            pasture_yes = pasture_widget.value 
            apply_fmask = fmask_widget.value
                
            # select aimag and soum
            soum_aimag = fc.filter(ee.Filter.And(ee.Filter.eq("aimag_eng", province_id), ee.Filter.eq("soum_eng", soum_id)))
            layer_name = str(province_id) + ' ' + str(soum_id)
            geom = soum_aimag.geometry()
            
#             Map.addLayer(ee.Image().paint(geom, 0, 2), {'palette': 'red'}, layer_name)
            
            if nd_indices_id == 'Normalized Difference Vegetation Index (NDVI)':
                ndviCollection = yearlyComposite(2014, 2021, geom).select('ndvi')
            elif nd_indices_id == 'Normalized Difference Water Index (NDWI)':
                ndviCollection = yearlyComposite(2014, 2021, geom).select('ndwi')
            elif nd_indices_id == 'Modified Soil Adjusted Vegetation Index (MSAVI)':
                ndviCollection = yearlyComposite(2014, 2021, geom).select('msavi')
            elif nd_indices_id == 'Enhanced Vegetation Index (EVI)':
                ndviCollection = yearlyComposite(2014, 2021, geom).select('evi')
            elif nd_indices_id == 'Near-Infrared Reflectence Vegetation (NIRv)':
                ndviCollection = yearlyComposite(2014, 2021, geom).select('nirv')
            
            
            dateIni = ee.Date.fromYMD(selected_year, 1, 1)
            dateEnd = ee.Date.fromYMD(selected_year, 12, 31)
            
            selected_image = ndviCollection.filterDate(dateIni, dateEnd).first()
            

            yMean = ndviCollection.mean()
            stdImg = ndviCollection.reduce(ee.Reducer.stdDev())
            
            Anomaly = selected_image.subtract(yMean).divide(stdImg).clip(geom)


            colorizedVis = {
                'min': 0.0,
                'max': 1.0,
                'palette': [
                    'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
                    '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
                    '012E01', '011D01', '011301'
                ]}
            
            anomParams = {'min': -3, 'max':3, 'palette': ['red', 'yellow', 'green']}
            
            anom_layer_name = 'Landsat 8' + ': ' + str(selected_year) + ' ' + str(nd_indices_id)
            
            Map.addLayer(Anomaly, anomParams, anom_layer_name)
            Map.center_object(soum_aimag, 9)
#             Map.remove_ee_layer(anom_layer_name)
            
            Map.default_style = {'cursor': 'default'}
            
        except Exception as e:
            print(e)
            print('An error occurred during computation.')
            
submit.on_click(submit_clicked)

## Drought

In [21]:
out = widgets.Output(layout={'border': '1px solid black'})
display(out)

Output(layout=Layout(border='1px solid black'))

In [22]:
def submit_clicked_chart(a):
    with out:
        out.clear_output()
        try:
            pdsi = ee.ImageCollection('IDAHO_EPSCOR/TERRACLIMATE').filterDate(date_range).select('pdsi')
            # aoi = ee.FeatureCollection('EPA/Ecoregions/2013/L3').filter(
            #     ee.Filter.eq('na_l3name', 'Sierra Nevada')).geometry()
            # pdsi = ee.ImageCollection('GRIDMET/DROUGHT').select('pdsi')

            # define area of interest
            aoi = fc.filter(ee.Filter.And(ee.Filter.eq("aimag_eng", province.value), ee.Filter.eq("soum_eng", soum.value))).geometry()
            # aoi = ee.FeatureCollection(fc).filter(ee.Filter.eq('asid', 8443)).geometry()
            # aoi_subset_collection = ee.FeatureCollection(fc).filter(ee.Filter.inList('asid', ee.List([4173, 4176])))

            # Reduce data

            # 1. Create a region reduction function.
            # 2. Map the function over the `pdsi` image collection to reduce each image.
            # 3. Filter out any resulting features that have null computed values (occurs when all pixels in an AOI are masked).
            reduce_pdsi = create_reduce_region_function(
                geometry=aoi, reducer=ee.Reducer.mean(), scale=5000, crs='EPSG:4326')

            pdsi_stat_fc = ee.FeatureCollection(pdsi.map(reduce_pdsi)).filter(
                ee.Filter.notNull(pdsi.first().bandNames()))

            # Server to client transfer
            pdsi_dict = fc_to_dict(pdsi_stat_fc).getInfo()

            # convert to Panda dataframe
            pdsi_df = pd.DataFrame(pdsi_dict)

            # add date variables to Dataframe function to the Dataframe
            pdsi_df = add_date_info(pdsi_df)

            # more cleaning
            pdsi_df = pdsi_df.rename(columns={
                'pdsi': 'PDSI'
            }).drop(columns=['millis', 'system:index'])

            pdsi_df['PDSI'] = pdsi_df['PDSI'] / 100

            # calendar map
            a = alt.Chart(pdsi_df).mark_rect().encode(
                x='Year:O',
                y='Month:O',
                color=alt.Color(
                    'mean(PDSI):Q', scale=alt.Scale(scheme='redblue', domain=(-10, 10))),
                tooltip=[
                    alt.Tooltip('Year:O', title='Year'),
                    alt.Tooltip('Month:O', title='Month'),
                    alt.Tooltip('mean(PDSI):Q', title='PDSI')
                ]).properties(width=600, height=300)

            a.display()
            
            b = alt.Chart(pdsi_df).mark_bar(size=3).encode(
                x='Timestamp:T',
                y='PDSI:Q',
                color=alt.Color(
                    'PDSI:Q', scale=alt.Scale(scheme='redblue', domain=(-10, 10))),
                tooltip=[
                    alt.Tooltip('Timestamp:T', title='Date'),
                    alt.Tooltip('PDSI:Q', title='PDSI')
                ]).properties(width=600, height=300)
            
            b.display()
            
        except Exception as e:
            print(e)
            print('An error occurred during computation.')

submit.on_click(submit_clicked_chart)

## Vegetation productivity

In [23]:
out_chart_2 = widgets.Output(layout = {'border': '1px solid black'})
display(out_chart_2)

Output(layout=Layout(border='1px solid black'))

In [24]:
def submit_click_chart2(a):
    with out_chart_2:
        out_chart_2.clear_output()
        try:
            ndvi = ee.ImageCollection('MODIS/006/MOD13A2').filterDate(date_range).select('NDVI')
            
            # define area of interest as it changes from widget boxes
            aoi = fc.filter(ee.Filter.And(ee.Filter.eq("aimag_eng", province.value), ee.Filter.eq("soum_eng", soum.value))).geometry()
            
            # prepare for reducing the available images
            reduce_ndvi = create_reduce_region_function(
                geometry = aoi, reducer = ee.Reducer.mean(), scale = 1000, crs = 'EPSG: 4326')
            
            # apply reduction function to produce FeatureCollection
            ndvi_stat_fc = ee.FeatureCollection(ndvi.map(reduce_ndvi)).filter(
                ee.Filter.notNull(ndvi.first().bandNames()))
            
            # Server to client converstion
            ndvi_dict = fc_to_dict(ndvi_stat_fc).getInfo()
            
            # read in Panda dataframe
            ndvi_df = pd.DataFrame(ndvi_dict)
            
            # some cleaning and add dates
            ndvi_df['NDVI'] = ndvi_df['NDVI'] / 10000
            ndvi_df = add_date_info(ndvi_df)
            
            # graphing
            highlight = alt.selection(
                type = 'single', on = 'mouseover', fields = ['Year'], nearest = True)
            
            base = alt.Chart(ndvi_df).encode(
                x=alt.X('DOY:Q', scale=alt.Scale(domain=[0, 350], clamp=True)),
                y=alt.Y('NDVI:Q', scale=alt.Scale(domain=[0.00, max(ndvi_df['NDVI'])])),
                color=alt.Color('Year:O', scale=alt.Scale(scheme='magma')))

            points = base.mark_circle().encode(
                opacity=alt.value(0),
                tooltip=[
                    alt.Tooltip('Year:O', title='Year'),
                    alt.Tooltip('DOY:Q', title='DOY'),
                    alt.Tooltip('NDVI:Q', title='NDVI')
                ]).add_selection(highlight)

            lines = base.mark_line().encode(
                size=alt.condition(~highlight, alt.value(1), alt.value(3)))
            
            a = (points + lines).properties(width = 600, height = 350).interactive()
            
            a.display()
            
            base2 = alt.Chart(ndvi_df).encode(
                x=alt.X('DOY:Q', scale=alt.Scale(domain=(0, 350))))

            line2 = base2.mark_line().encode(
                y=alt.Y('median(NDVI):Q', scale=alt.Scale(domain=(0.00, max(ndvi_df['NDVI'])))))

            band2 = base2.mark_errorband(extent='iqr').encode(
                y='NDVI:Q')

            b = (line2 + band2).properties(width=600, height=300).interactive()
            
            b.display()
        
        except Exception as e:
            print(e)
            print('An error occurred during computation.')

submit.on_click(submit_click_chart2)

In [25]:
## Dought and productivity relationship
# ndvi_doy_range = [224, 272]

# ndvi_df_sub = ndvi_df[(ndvi_df['DOY'] >= ndvi_doy_range[0])
#                       & (ndvi_df['DOY'] <= ndvi_doy_range[1])]

# ndvi_df_sub = ndvi_df_sub.groupby('Year').agg('min')

In [26]:
# pdsi_doy_range = [1, 272]

# pdsi_df_sub = pdsi_df[(pdsi_df['DOY'] >= pdsi_doy_range[0])
#                       & (pdsi_df['DOY'] <= pdsi_doy_range[1])]

# pdsi_df_sub = pdsi_df_sub.groupby('Year').agg('mean')

In [27]:
# ndvi_pdsi_df = pd.merge(
#     ndvi_df_sub, pdsi_df_sub, how='left', on='Year').reset_index()

# ndvi_pdsi_df = ndvi_pdsi_df[['Year', 'NDVI', 'PDSI']]

In [28]:
# ndvi_pdsi_df = ndvi_pdsi_df.dropna()

In [29]:
# ndvi_pdsi_df['Fit'] = np.poly1d(
#     np.polyfit(ndvi_pdsi_df['PDSI'], ndvi_pdsi_df['NDVI'], 1))(
#         ndvi_pdsi_df['PDSI'])

In [30]:
# pd.notnull(ndvi_pdsi_df)

In [31]:
# base = alt.Chart(ndvi_pdsi_df).encode(
#     x=alt.X('PDSI:Q', scale=alt.Scale(domain=(-6.5, 1.5))))

# points = base.mark_circle(size=60).encode(
#     y=alt.Y('NDVI:Q', scale=alt.Scale(domain=(0.1, 0.26))),
#     color=alt.Color('Year:O', scale=alt.Scale(scheme='magma')),
#     tooltip=[
#         alt.Tooltip('Year:O', title='Year'),
#         alt.Tooltip('PDSI:Q', title='PDSI'),
#         alt.Tooltip('NDVI:Q', title='NDVI')
#     ])

# fit = base.mark_line().encode(
#     y=alt.Y('Fit:Q'),
#     color=alt.value('#808080'))

# (points + fit).properties(width=600, height=300).interactive()

## Past and future climate

In [32]:
out_chart_3 = widgets.Output(layout = {'border': '1px solid black'})
display(out_chart_3)

Output(layout=Layout(border='1px solid black'))

In [33]:
def submit_click_chart3(a):
    with out_chart_3:
        out_chart_3.clear_output()
        try:
            #### Future climate
            startDate = '2020-01-01'
            endDate = '2025-12-31'

            monthDifference = ee.Date(startDate).advance(1, 'month').millis().subtract(ee.Date(startDate).millis())
            listMap = ee.List.sequence(ee.Date(startDate).millis(), ee.Date(endDate).millis(), monthDifference)

            def monthly_temp_composite(dateMillis):
                date = ee.Date(dateMillis)
                nasa_future = (ee.ImageCollection('NASA/NEX-GDDP')
                              .select(['tasmax', 'tasmin', 'pr'])
                              .filter(ee.Filter.eq('scenario', 'rcp85'))
                              .filterDate(date, date.advance(1, 'month'))
                              .mean()
                              .set('system:index', date.format())
                              .set('system:time_start', date.millis()) )                  
                return nasa_future

            nasa_future_climate = ee.ImageCollection.fromImages(listMap.map(monthly_temp_composite))
            # nasa_future_month_pr = ee.ImageCollection.fromImages(listMap.map(monthly_pr_composite))

            # nasa_future_climate = nasa_future_month_temp.combine(nasa_future_month_pr)


            def calc_mean_temp(img):
                return (img.select('tasmax') \
                        .add(img.select('tasmin')) \
                        .divide(ee.Image.constant(2.0)) \
                        .addBands(img.select('pr')) \
                        .rename(['Temp-mean', 'Precip-rate']) \
                        .copyProperties(img, img.propertyNames()))

            nasa_future_month = nasa_future_climate.map(calc_mean_temp)
            
            # define area of interest as it changes from widget boxes
            aoi = fc.filter(ee.Filter.And(ee.Filter.eq("aimag_eng", province.value), ee.Filter.eq("soum_eng", soum.value))).geometry()
            
            reduce_future_climate = create_reduce_region_function(
                geometry=aoi, reducer=ee.Reducer.mean(), scale=5000, crs='EPSG:4326')

            future_climate_stat_fc = ee.FeatureCollection(nasa_future_month.map(reduce_future_climate)).filter(
                ee.Filter.notNull(nasa_future_month.first().bandNames()))

            dcp_dict = fc_to_dict(future_climate_stat_fc).getInfo()
            dcp_df = pd.DataFrame(dcp_dict)

            # add dates
            dcp_df = add_date_info(dcp_df)

            # do some cleaning
            dcp_df['Precip-mm'] = dcp_df['Precip-rate'] * 86400 * 30
            dcp_df['Temp-mean'] = dcp_df['Temp-mean'] - 273.15
            dcp_df['Model'] = 'NEX-GDDP'
            dcp_df = dcp_df.drop('Precip-rate', 1)

            ##############################      Past CLIMATE      #################################3
            
            terra_climate = (ee.ImageCollection('IDAHO_EPSCOR/TERRACLIMATE')
                             .filter(ee.Filter.date('1980-01-01', '2021-01-01'))
                             .select(['tmmn', 'tmmx', 'pr']))

            reduce_era5 = create_reduce_region_function(
                geometry=aoi, reducer=ee.Reducer.mean(), scale=5000, crs='EPSG:4326')

            era5_stat_fc = (ee.FeatureCollection(terra_climate.map(reduce_era5))
                             .filter(ee.Filter.notNull(terra_climate.first().bandNames())))

            # Server to Client
            era5_dict = fc_to_dict(era5_stat_fc).getInfo()

            # Panda data frame
            era5_df = pd.DataFrame(era5_dict)

            # do some cleaning
            era5_df = add_date_info(era5_df)
            era5_df['Model'] = 'TERRA_CLIMATE'
            era5_df['Temp-mean'] = (era5_df['tmmn'] + era5_df['tmmx'])/2*0.1
            era5_df = era5_df.rename(columns={'pr': 'Precip-mm'})
            era5_df = era5_df.drop(['tmmn', 'tmmx'], 1)

            ############################## Combine #########################################

            # combine past and future climate data
            climate_df = pd.concat([era5_df, dcp_df], sort=True)

            # chart 1
            base = alt.Chart(climate_df).encode(
                x='Year:O',
                color='Model')

            line = base.mark_line().encode(
                y=alt.Y('median(Precip-mm):Q', title='Precipitation (mm/month)'))

            band = base.mark_errorband(extent='iqr').encode(
                y=alt.Y('Precip-mm:Q', title='Precipitation (mm/month)'))

            a = (band + line).properties(width=600, height=300)

            a.display()

            # chart 2
            line2 = alt.Chart(climate_df).mark_line().encode(
                x='Year:O',
                y='median(Temp-mean):Q',
                color='Model')

            band2 = alt.Chart(climate_df).mark_errorband(extent='iqr').encode(
                x='Year:O',
                y=alt.Y('Temp-mean:Q', title='Temperature (°C)'), color='Model')

            b = (band2 + line2).properties(width=600, height=300)

            b.display()
            
        except Exception as e:
            print(e)
            print('An error occurred during computation.')
            
submit.on_click(submit_click_chart3)

In [34]:
# #### Future climate
# startDate = '2020-01-01'
# endDate = '2028-12-31'

# monthDifference = ee.Date(startDate).advance(1, 'month').millis().subtract(ee.Date(startDate).millis())
# listMap = ee.List.sequence(ee.Date(startDate).millis(), ee.Date(endDate).millis(), monthDifference)

# def monthly_temp_composite(dateMillis):
#     date = ee.Date(dateMillis)
#     nasa_future = (ee.ImageCollection('NASA/NEX-GDDP')
#                   .select(['tasmax', 'tasmin', 'pr'])
#                   .filter(ee.Filter.eq('scenario', 'rcp85'))
#                   .filterDate(date, date.advance(1, 'month'))
#                   .mean()
#                   .set('system:index', date.format())
#                   .set('system:time_start', date.millis()) )                  
#     return nasa_future
    
# nasa_future_climate = ee.ImageCollection.fromImages(listMap.map(monthly_temp_composite))
# # nasa_future_month_pr = ee.ImageCollection.fromImages(listMap.map(monthly_pr_composite))

# # nasa_future_climate = nasa_future_month_temp.combine(nasa_future_month_pr)


# def calc_mean_temp(img):
#     return (img.select('tasmax') \
#             .add(img.select('tasmin')) \
#             .divide(ee.Image.constant(2.0)) \
#             .addBands(img.select('pr')) \
#             .rename(['Temp-mean', 'Precip-rate']) \
#             .copyProperties(img, img.propertyNames()))

# nasa_future_month = nasa_future_climate.map(calc_mean_temp)

# reduce_future_climate = create_reduce_region_function(
#     geometry=aoi, reducer=ee.Reducer.mean(), scale=5000, crs='EPSG:4326')

# future_climate_stat_fc = ee.FeatureCollection(nasa_future_month.map(reduce_future_climate)).filter(
#     ee.Filter.notNull(nasa_future_month.first().bandNames()))

# dcp_dict = fc_to_dict(future_climate_stat_fc).getInfo()
# dcp_df = pd.DataFrame(dcp_dict)

# # add dates
# dcp_df = add_date_info(dcp_df)

# # do some cleaning
# dcp_df['Precip-mm'] = dcp_df['Precip-rate'] * 86400 * 30
# dcp_df['Temp-mean'] = dcp_df['Temp-mean'] - 273.15
# dcp_df['Model'] = 'NEX-GDDP'
# dcp_df = dcp_df.drop('Precip-rate', 1)

# ##############################      Past CLIMATE      #################################3
# terra_climate = (ee.ImageCollection('IDAHO_EPSCOR/TERRACLIMATE')
#                  .filter(ee.Filter.date('1980-01-01', '2021-01-01'))
#                  .select(['tmmn', 'tmmx', 'pr']))

# reduce_era5 = create_reduce_region_function(
#     geometry=aoi, reducer=ee.Reducer.mean(), scale=5000, crs='EPSG:4326')

# era5_stat_fc = (ee.FeatureCollection(terra_climate.map(reduce_era5))
#                  .filter(ee.Filter.notNull(terra_climate.first().bandNames())))

# # Server to Client
# era5_dict = fc_to_dict(era5_stat_fc).getInfo()

# # Panda data frame
# era5_df = pd.DataFrame(era5_dict)

# # do some cleaning
# era5_df = add_date_info(era5_df)
# era5_df['Model'] = 'TERRA_CLIMATE'
# era5_df['Temp-mean'] = (era5_df['tmmn'] + era5_df['tmmx'])/2*0.1
# era5_df = era5_df.rename(columns={'pr': 'Precip-mm'})
# era5_df = era5_df.drop(['tmmn', 'tmmx'], 1)

# ############################## Combine #########################################

# # combine past and future climate data
# climate_df = pd.concat([era5_df, dcp_df], sort=True)

# # chart 1
# base = alt.Chart(climate_df).encode(
#     x='Year:O',
#     color='Model')

# line = base.mark_line().encode(
#     y=alt.Y('median(Precip-mm):Q', title='Precipitation (mm/month)'))

# band = base.mark_errorband(extent='iqr').encode(
#     y=alt.Y('Precip-mm:Q', title='Precipitation (mm/month)'))

# a = (band + line).properties(width=600, height=300)

# a.display()

# # chart 2
# line2 = alt.Chart(climate_df).mark_line().encode(
#     x='Year:O',
#     y='median(Temp-mean):Q',
#     color='Model')

# band2 = alt.Chart(climate_df).mark_errorband(extent='iqr').encode(
#     x='Year:O',
#     y=alt.Y('Temp-mean:Q', title='Temperature (°C)'), color='Model')

# b = (band2 + line2).properties(width=600, height=300)

# b.display()

In [35]:
# nasa_future_month.first().getInfo()

In [36]:
# reduce_future_climate = create_reduce_region_function(
#     geometry=aoi, reducer=ee.Reducer.mean(), scale=5000, crs='EPSG:4326')

# future_climate_stat_fc = ee.FeatureCollection(nasa_future_month.map(reduce_future_climate)).filter(
#     ee.Filter.notNull(nasa_future_month.first().bandNames()))

In [37]:
# future_climate_stat_fc.first().getInfo()

In [38]:
# dcp_stat_fc.first().getInfo()

In [39]:
# task = ee.batch.Export.table.toAsset(
#     collection=dcp_stat_fc,
#     description='future_climate_stat_fc export',
#     assetId='future_climate_stat_fc')

# task.start()

In [40]:
# dcp_dict = fc_to_dict(future_climate_stat_fc).getInfo()
# dcp_df = pd.DataFrame(dcp_dict)

# # add dates
# dcp_df = add_date_info(dcp_df)

# # do some cleaning
# dcp_df['Precip-mm'] = dcp_df['Precip-rate'] * 86400 * 30
# dcp_df['Temp-mean'] = dcp_df['Temp-mean'] - 273.15
# dcp_df['Model'] = 'NEX-GDDP'
# dcp_df = dcp_df.drop('Precip-rate', 1)

In [41]:
# #### Past CLIMATE
# terra_climate = (ee.ImageCollection('IDAHO_EPSCOR/TERRACLIMATE')
#                  .filter(ee.Filter.date('1980-01-01', '2021-01-01'))
#                  .select(['tmmn', 'tmmx', 'pr']))

# reduce_era5 = create_reduce_region_function(
#     geometry=aoi, reducer=ee.Reducer.mean(), scale=5000, crs='EPSG:4326')

# era5_stat_fc = (ee.FeatureCollection(terra_climate.map(reduce_era5))
#                  .filter(ee.Filter.notNull(terra_climate.first().bandNames())))

# # Server to Client
# era5_dict = fc_to_dict(era5_stat_fc).getInfo()

# # Panda data frame
# era5_df = pd.DataFrame(era5_dict)

# # do some cleaning
# era5_df = add_date_info(era5_df)
# era5_df['Model'] = 'TERRA_CLIMATE'
# era5_df['Temp-mean'] = (era5_df['tmmn'] + era5_df['tmmx'])/2*0.1
# era5_df = era5_df.rename(columns={'pr': 'Precip-mm'})
# era5_df = era5_df.drop(['tmmn', 'tmmx'], 1)

# # combine past and future climate data
# climate_df = pd.concat([era5_df, dcp_df], sort=True)

In [42]:
# base = alt.Chart(climate_df).encode(
#     x='Year:O',
#     color='Model')

# line = base.mark_line().encode(
#     y=alt.Y('median(Precip-mm):Q', title='Precipitation (mm/month)'))

# band = base.mark_errorband(extent='iqr').encode(
#     y=alt.Y('Precip-mm:Q', title='Precipitation (mm/month)'))

# (band + line).properties(width=600, height=300)

In [43]:
# line = alt.Chart(climate_df).mark_line().encode(
#     x='Year:O',
#     y='median(Temp-mean):Q',
#     color='Model')

# band = alt.Chart(climate_df).mark_errorband(extent='iqr').encode(
#     x='Year:O',
#     y=alt.Y('Temp-mean:Q', title='Temperature (°C)'), color='Model')

# (band + line).properties(width=600, height=300)